In [10]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.embeddings import OctoAIEmbeddings
from langchain_community.vectorstores import Milvus
import os

In [11]:
files = os.listdir("./data")
embeddings = OctoAIEmbeddings(endpoint_url="https://text.octoai.run/v1/embeddings")

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
file_texts = []
for file in files:
    with open(f"./data/{file}") as f:
        file_text = f.read()
    # text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    #     chunk_size=512, chunk_overlap=64, 
    # )

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)
    texts = text_splitter.split_text(file_text)
    for i, chunked_text in enumerate(texts):
        print(f"\n-----{len(chunked_text)}----" + chunked_text)
        file_texts.append(Document(page_content=chunked_text, 
                metadata={"doc_title": file.split(".")[0], "chunk_num": i}))
        #break


-----511----Well, good afternoon, ladies and gentlemen. My name is Art Benjamin, and I am a mathemagician. What that means is I combine my loves of math, or I should say, maths, and magic to do what I call mathemagics. But before I get started, I've got a quick question for the audience.  By any chance, did anyone here in the audience happen to bring with them this afternoon a calculator? If you have one, perhaps on your phone or somewhere, and you're pretty comfortable using it, raise your hand. I'll need a couple of

-----511----using it, raise your hand. I'll need a couple of people to help me out here. I see one, two, and perhaps one more. Three. Would the three of you bring out your calculators and join me up here on stage, and let's give these volunteers a nice round of applause. Come on up.  Great. Over on this side, please. Awesome. Now, since I have not had the chance to work with these calculators, I need to make sure that they are all working properly. Would somebody get us

In [7]:

vector_store = Milvus.from_documents(
    file_texts,
    embedding=embeddings,
    connection_args={"host": "localhost", "port": 19530},
    collection_name="transcription"
)

In [15]:
retriever = vector_store.as_retriever()

In [12]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms.octoai_endpoint import OctoAIEndpoint

In [13]:
llm = OctoAIEndpoint(
    endpoint_url="https://text.octoai.run/v1/chat/completions",
    model_kwargs={
        "model": "mixtral-8x7b-instruct-fp16",
        "max_tokens": 128,
        "presence_penalty": 0,
        "temperature": 0.01,
        "top_p": 0.9,
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant. Keep your responses limited to one short paragraph if possible.",
            },
        ],
    },
)

In [14]:
from langchain_community.embeddings import OctoAIEmbeddings
from langchain_community.vectorstores import Milvus

In [15]:
embeddings = OctoAIEmbeddings(endpoint_url="https://text.octoai.run/v1/embeddings")

In [16]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
import os

In [31]:
from octoai.client import Client
import base64
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["OCTOAI_API_TOKEN"] = os.getenv("OCTOAI_API_TOKEN")

whisper_url = "https://whisper-demo-kk0powt97tmb.octoai.run/predict"
whisper_health_check = "https://whisper-demo-kk0powt97tmb.octoai.run/healthcheck"

client = Client(token=os.getenv("OCTOAI_API_TOKEN"))


# Transcribe videos
def transcribe_video(video_dir: str, output_dir: str):
    # loop through videofiles in the directory
    for video_file in os.listdir(video_dir):
        video_path = os.path.join(video_dir, video_file)
        # open the video file and encode it to base64
        with open(video_path, "rb") as f:
            encoded_video = base64.b64encode(f.read())
            base64_string = encoded_video.decode("utf-8")
        # create the input dictionary
        inputs = {
            "language": "en",
            "task": "transcribe",
            "audio": base64_string,
        }
        # send the input dictionary to the endpoint
        if client.health_check(whisper_health_check) == 200:
            outputs = client.infer(endpoint_url=whisper_url, inputs=inputs)
            transcription = outputs["transcription"]
        # save the response to a file
        output_path = os.path.join(output_dir, video_file + ".txt")
        # create output direcotry if it does not exist
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        with open(output_path, "w") as f:
            f.write(transcription)
        print(f"Transcribed {video_file} to {output_path}")


transcribe_video("./videos", "./data")

Transcribed Faster than a calculator  Arthur Benjamin  TEDxOxford.mp3 to ./data\Faster than a calculator  Arthur Benjamin  TEDxOxford.mp3.txt


OctoAIClientError: Rate limit reached error: 429 {"message":"You've exceeded the maximum rate limit (15 requests per 1h0m0s) for this endpoint. Please contact our Customer Success team at customer-success@octoml.ai if you need to exceed this limit."}

In [8]:
def video_question(language, prompt):
    content = f"You are a helpful assistant who responds in {language} and not in English."
    llm = OctoAIEndpoint(
    endpoint_url="https://text.octoai.run/v1/chat/completions",
    model_kwargs={
        "model": "mixtral-8x7b-instruct-fp16",
        "max_tokens": 128,
        "presence_penalty": 0,
        "temperature": 0.1,
        "top_p": 0.9,
        "messages": [
            {
                "role": "system",
                "content": content,
            },
        ],
    },
    )

# Let's make this a bit more fun and showcase the multilingual capabilities of Mixtal which really outshine other open source models

# Our Vector DB is populated with entries from english text - even the embedding model we're using here, GTE-Large
# works best on english text. However Mixtral has good mutlilingual capabilities in French, German, Spanish and Italian.
# So what we'll do is ask the assistant to only answer in {french in the system and user prompt. RAG here is performed based on 
# english text, but upon producing the user response, the Mixtral LLM will generate tokens in a different language here (french)

    template = f"Answer the question in {language} only on the following context:"
    template += """"
    {context}
    
    Question: {question}
    """
    llm_prompt = PromptTemplate.from_template(template)

    chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | llm_prompt
    | llm
    | StrOutputParser()
)
    return chain.invoke(prompt)

In [9]:
print(video_question("French","What is the best way to learn math"))

NameError: name 'OctoAIEndpoint' is not defined

In [42]:
print(video_question("Chinese","What are some math tricks I should know"))

 在开始介绍具体的数学技巧之前，我想先回答您提出的问题。作为一个数学爱好者，有很多数学技巧可以学习和掌握。以下是一些您可能感兴趣的数学技巧：

1. 数字乘法：您可以学习一些快速计算数字乘法的技巧，例如，将两个两位数相乘时，可以将


In [5]:
print(video_question("English","What is the quickest way to learn math?"))

NameError: name 'video_question' is not defined

In [4]:
print(video_question("Russian","What are some math tricks I should know"))

NameError: name 'video_question' is not defined